In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import scipy.io
import math

# format: virtualflies x (PPL1_01, 02, .., 05, odor)
# arranged odor #0 - #4
virtualFly_PPL1 = scipy.io.loadmat('virtualFly_PPL1.mat')['virtualFly']
virtualFly_MBON = scipy.io.loadmat('virtualFly_MBON.mat')['virtualFly_MBON']

In [13]:
ODOR_NUM = 5
PPL1_NUM = 5
MBON_NUM = 6
NEURON_NUM = PPL1_NUM + MBON_NUM
NUM_FLIES_PER_ODOR = 12


VFP_OD1 = virtualFly_PPL1[:NUM_FLIES_PER_ODOR**PPL1_NUM,:]
VFP_OD2 = virtualFly_PPL1[NUM_FLIES_PER_ODOR**PPL1_NUM:2*NUM_FLIES_PER_ODOR**PPL1_NUM,:]
VFP_OD3 = virtualFly_PPL1[2*NUM_FLIES_PER_ODOR**PPL1_NUM:3*NUM_FLIES_PER_ODOR**PPL1_NUM,:]
VFP_OD4 = virtualFly_PPL1[3*NUM_FLIES_PER_ODOR**PPL1_NUM:4*NUM_FLIES_PER_ODOR**PPL1_NUM,:]
VFP_OD5 = virtualFly_PPL1[4*NUM_FLIES_PER_ODOR**PPL1_NUM:5*NUM_FLIES_PER_ODOR**PPL1_NUM,:]

np.random.shuffle(VFP_OD1)
np.random.shuffle(VFP_OD2)
np.random.shuffle(VFP_OD3)
np.random.shuffle(VFP_OD4)
np.random.shuffle(VFP_OD5)

VFM_OD1 = virtualFly_MBON[:NUM_FLIES_PER_ODOR**MBON_NUM,:]
VFM_OD2 = virtualFly_MBON[NUM_FLIES_PER_ODOR**MBON_NUM:2*NUM_FLIES_PER_ODOR**MBON_NUM,:]
VFM_OD3 = virtualFly_MBON[2*NUM_FLIES_PER_ODOR**MBON_NUM:3*NUM_FLIES_PER_ODOR**MBON_NUM,:]
VFM_OD4 = virtualFly_MBON[3*NUM_FLIES_PER_ODOR**MBON_NUM:4*NUM_FLIES_PER_ODOR**MBON_NUM,:]
VFM_OD5 = virtualFly_MBON[4*NUM_FLIES_PER_ODOR**MBON_NUM:5*NUM_FLIES_PER_ODOR**MBON_NUM,:]

np.random.shuffle(VFM_OD1)
np.random.shuffle(VFM_OD2)
np.random.shuffle(VFM_OD3)
np.random.shuffle(VFM_OD4)
np.random.shuffle(VFM_OD5)



In [25]:
#combining PPL1 + MBON

numSamples = VFP_OD1.shape[0]
VF_OD1 = np.concatenate((VFP_OD1[:,:PPL1_NUM], VFM_OD1[:numSamples,:]),axis=1)
VF_OD2 = np.concatenate((VFP_OD2[:,:PPL1_NUM], VFM_OD2[:numSamples,:]),axis=1)
VF_OD3 = np.concatenate((VFP_OD3[:,:PPL1_NUM], VFM_OD3[:numSamples,:]),axis=1)
VF_OD4 = np.concatenate((VFP_OD4[:,:PPL1_NUM], VFM_OD4[:numSamples,:]),axis=1)
VF_OD5 = np.concatenate((VFP_OD5[:,:PPL1_NUM], VFM_OD5[:numSamples,:]),axis=1)


In [26]:
# dataset arrange

# based on PPL1 size
numTrain = virtualFly_PPL1.shape[0] * 0.9
numVal = virtualFly_PPL1.shape[0] * 0.05
numTest = virtualFly_PPL1.shape[0] * 0.05
numTrainPerOD = math.floor(numTrain/ODOR_NUM)
numValPerOD = math.floor(numVal/ODOR_NUM)
numTestPerOD = math.floor(numTest/ODOR_NUM)
'''
print(numTrain - numTrainPerOD*ODOR_NUM)
print(numVal - numValPerOD*ODOR_NUM)
print(numTest - numTestPerOD*ODOR_NUM)
print(virtualFly.shape[0]/5 - numTrainPerOD - numValPerOD - numTestPerOD)
print(numTrainPerOD, numValPerOD, numTestPerOD)
'''
mask = range(numTrainPerOD)
trainSet = np.concatenate((VF_OD1[mask,:], VF_OD2[mask,:], VF_OD3[mask,:], VF_OD4[mask,:], VF_OD5[mask,:]),axis=0) 
np.random.shuffle(trainSet)
X_train = trainSet[:,:NEURON_NUM]
y_train = trainSet[:,-1]-1 # -1 to conform python index from 0 to 4 (class imported from MATLAB was from 1 to 5)
X_train_shuffle = np.copy(X_train)
np.random.shuffle(X_train_shuffle)

mask = range(numTrainPerOD, numTrainPerOD+numValPerOD)
valSet = np.concatenate((VF_OD1[mask,:], VF_OD2[mask,:], VF_OD3[mask,:], VF_OD4[mask,:], VF_OD5[mask,:]),axis=0) 
np.random.shuffle(valSet)
X_val = valSet[:,:NEURON_NUM]
y_val = valSet[:,-1]-1 # to conform python index format

mask = range(numTrainPerOD+numValPerOD, numTrainPerOD+numValPerOD+numTestPerOD)
testSet = np.concatenate((VF_OD1[mask,:], VF_OD2[mask,:], VF_OD3[mask,:], VF_OD4[mask,:], VF_OD5[mask,:]),axis=0) 
np.random.shuffle(testSet)
X_test = testSet[:,:NEURON_NUM]
y_test = testSet[:,-1]-1 # to conform python index format

print(numTrain, trainSet.shape)
print(numVal, valSet.shape)
print(numTest, testSet.shape)

print('Train shape: ', X_train.shape)
print('Validation shape: ', X_val.shape)
print('Test shape: ', X_test.shape)

1119744.0 (1119740, 12)
62208.0 (62205, 12)
62208.0 (62205, 12)
Train shape:  (1119740, 11)
Validation shape:  (62205, 11)
Test shape:  (62205, 11)


In [27]:
# preprocessing

#-mean - should do std as well?
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
print(mean,'\n', std)

X_train -= mean
X_train_shuffle -= mean
X_val -= mean
X_test -= mean

'''
X_train /= std
X_val /= std
X_test /= std
'''

[ 1.20979881  7.01952799 -0.05275849  2.77959281  8.1569809  18.50519541
  5.17610824  5.768908    7.7611272  11.75774726 11.03249513] 
 [ 6.55190598  6.6805451   5.11444378  6.89041502  3.84017398 13.52565991
  9.80513404  7.38232189  8.4672489   7.56104869 12.28224976]


'\nX_train /= std\nX_val /= std\nX_test /= std\n'

In [32]:
def train(X, Y, XV, YV, model, lr, reg, batchSize, epoch):
    X = torch.Tensor(X)
    Y = torch.Tensor(Y)
    #X = torch.FloatTensor(X)
    #Y = torch.FloatTensor(Y)
    N = len(Y)
    XV = torch.FloatTensor(XV)
    YV = torch.FloatTensor(YV)

    #optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    optimizer = optim.Adam(model.parameters(), lr=lr)


   
    for epoch in range(epoch):
        perm = torch.randperm(N)
        sum_loss = 0
        model.train()
        for i in range(0, N, batchSize):
            x = X[perm[i:i+batchSize]]
            y = Y[perm[i:i+batchSize]]

            optimizer.zero_grad()
            weight = model.weight
            bias = model.bias
            output = model(x)
            
            #print(weight)
            #print(bias)
            #print(x)
            #print(x.double() @ weight + bias.double())
            #print(torch.mm(x, torch.t(model.weight))
            #print(torch.addmm(bias, x, torch.t(weight)))
            
            '''
            print("x:",x)
            print("y:",y.long())
            print("output:",output)
            '''
            
            correct_class_score = output[torch.arange(x.shape[0]), y.long()]
            
            margin = torch.maximum(torch.zeros(output.shape), output - correct_class_score.unsqueeze(1) + 1) # delta is 1
            loss = (torch.sum(margin)-1*x.shape[0])/x.shape[0] # remove correct case when delta is 1
            #print("loss1:", loss)
            loss += reg * torch.sum((weight.t() @ weight)) #/ 2.0
            #print("loss2:", loss)
            
            '''
            print(output - correct_class_score.unsqueeze(1)+1)
            print(torch.zeros(output.shape))
            print(correct_class_score)
            print("loss:", loss)
            '''
            #loss = torch.mean(torch.clamp(1 - y * output, min=0))
            
            loss.backward()
            optimizer.step()

            sum_loss += float(loss)
            

        print("Epoch: {:4d}\tloss: {}".format(epoch, sum_loss / N))

        numScore = 0
        model.eval()
        for i in range(0, len(YV), batchSize):
            x = XV[i:i+batchSize]
            y = YV[i:i+batchSize]
            
            optimizer.zero_grad()
            with torch.no_grad():
                output = model(x).numpy()
                pred = np.argmax(output, axis = 1)
                #print(pred)
                #print(y.numpy())
                numScore += np.sum((pred == y.numpy()))
                #print(numScore)
            
        
        print("validation acuuracy: {}".format(numScore / len(YV)))
        
def test(X, Y, model, batchSize, numOnly = False):
    XT = torch.FloatTensor(X)
    YT = torch.FloatTensor(Y)
    numScore = 0
    model.eval()
    for i in range(0, len(YT), batchSize):
        x = XT[i:i+batchSize]
        y = YT[i:i+batchSize]
        output = model(x).detach().numpy()
        pred = np.argmax(output, axis = 1)
        #print(pred)
        #print(y.numpy())
        numScore += np.sum((pred == y.numpy()))
        #print(numScore)
    if numOnly:
        print(numScore / len(YT))
    else:
        print("test acuuracy: {}".format(numScore / len(YT)))


        
def visualize(X, Y, model):
    W = model.weight.squeeze().detach().cpu().numpy()
    b = model.bias.squeeze().detach().cpu().numpy()

    delta = 0.001
    x = np.arange(X[:, 0].min(), X[:, 0].max(), delta)
    y = np.arange(X[:, 1].min(), X[:, 1].max(), delta)
    x, y = np.meshgrid(x, y)
    xy = list(map(np.ravel, [x, y]))

    z = (W.dot(xy) + b).reshape(x.shape)
    z[np.where(z > 1.0)] = 4
    z[np.where((z > 0.0) & (z <= 1.0))] = 3
    z[np.where((z > -1.0) & (z <= 0.0))] = 2
    z[np.where(z <= -1.0)] = 1

    plt.figure(figsize=(10, 10))
    plt.xlim([X[:, 0].min() + delta, X[:, 0].max() - delta])
    plt.ylim([X[:, 1].min() + delta, X[:, 1].max() - delta])
    plt.contourf(x, y, z, alpha=0.8, cmap="Greys")
    plt.scatter(x=X[:, 0], y=X[:, 1], c="black", s=10)
    plt.tight_layout()
    plt.show()



In [29]:
lr = 0.0005
reg = 0.0001

batchSize = 200
epoch = 10
    
model = nn.Linear(11, 5)

train(X_train, y_train, X_val, y_val, model, lr, reg, batchSize, epoch)
test(X_test, y_test, model, batchSize)



Epoch:    0	loss: 0.005266073258307197
validation acuuracy: 0.8140181657423037
Epoch:    1	loss: 0.0025509063469651813
validation acuuracy: 0.8167349891487823
Epoch:    2	loss: 0.0024806035194266324
validation acuuracy: 0.8176352383248935
Epoch:    3	loss: 0.0024649123392121797
validation acuuracy: 0.8167832167832167
Epoch:    4	loss: 0.0024598902128346064
validation acuuracy: 0.8175870106904589
Epoch:    5	loss: 0.002458084295791491
validation acuuracy: 0.8172172654931276
Epoch:    6	loss: 0.002457498122510817
validation acuuracy: 0.8172654931275621
Epoch:    7	loss: 0.0024574468124208136
validation acuuracy: 0.8160598022666988
Epoch:    8	loss: 0.002457175517654126
validation acuuracy: 0.8170404308335343
Epoch:    9	loss: 0.0024569876315696476
validation acuuracy: 0.8170243549553894
test acuuracy: 0.8153846153846154


In [31]:
print("Shuffle Begins")
model_shuffle = nn.Linear(11,5);
train(X_train_shuffle, y_train, X_val, y_val, model_shuffle, lr, reg, batchSize, epoch)
test(X_test, y_test, model_shuffle, batchSize)

Shuffle Begins
Epoch:    0	loss: 0.021941065794443815
validation acuuracy: 0.1725745518848967
Epoch:    1	loss: 0.020003890174863544
validation acuuracy: 0.16159472711196848
Epoch:    2	loss: 0.020007618134899045
validation acuuracy: 0.22618760549795033
Epoch:    3	loss: 0.02000186104376479
validation acuuracy: 0.21213728799935697
Epoch:    4	loss: 0.02000156281741222
validation acuuracy: 0.20196125713367094
Epoch:    5	loss: 0.019999414320738318
validation acuuracy: 0.24314765694076038
Epoch:    6	loss: 0.020002845487181193
validation acuuracy: 0.12463628325697292
Epoch:    7	loss: 0.020005708114506422
validation acuuracy: 0.1628164938509766
Epoch:    8	loss: 0.020004477720354137
validation acuuracy: 0.15963346997829755
Epoch:    9	loss: 0.020011596193200663
validation acuuracy: 0.2561048147255044
test acuuracy: 0.2544650751547303


In [33]:
print(valSet.shape)
print(testSet.shape)
fullTestSet = np.concatenate((valSet,testSet),axis=0)
print(fullTestSet.shape)
np.random.shuffle(fullTestSet)
X_fullTest = fullTestSet[:,:NEURON_NUM]
y_fullTest = fullTestSet[:,-1]-1 # to conform python index format
print(X_fullTest.shape)
print(y_fullTest.shape)

(62205, 12)
(62205, 12)
(124410, 12)
(124410, 11)
(124410,)


In [36]:
mask = range(numTrainPerOD, numTrainPerOD+numValPerOD+numTestPerOD)
fullTestSet2 = np.concatenate((VF_OD1[mask,:], VF_OD2[mask,:], VF_OD3[mask,:], VF_OD4[mask,:], VF_OD5[mask,:]),axis=0) 

#np.random.shuffle(fullTestSet2)
X_val = valSet[:,:NEURON_NUM]
y_val = valSet[:,-1]-1 # to conform python index format

X_fullTestSet2 = testSet[:,:NEURON_NUM]
y_fullTestSet2 = testSet[:,-1]-1 # to conform python index format

In [34]:
sampleSize = 1000

#test for 120 cases
for i in range (0,120,1):
    test(X_fullTest[i*sampleSize:(i+1)*sampleSize,:], y_fullTest[i*sampleSize:(i+1)*sampleSize], model, batchSize, True)

0.822
0.816
0.813
0.801
0.833
0.815
0.82
0.825
0.812
0.809
0.827
0.804
0.816
0.809
0.827
0.823
0.821
0.827
0.803
0.807
0.81
0.785
0.824
0.816
0.8
0.821
0.811
0.811
0.82
0.837
0.842
0.818
0.812
0.827
0.811
0.814
0.829
0.815
0.818
0.813
0.818
0.787
0.81
0.823
0.83
0.836
0.812
0.801
0.828
0.786
0.819
0.838
0.824
0.813
0.828
0.813
0.813
0.814
0.821
0.82
0.833
0.826
0.803
0.811
0.833
0.807
0.835
0.828
0.804
0.821
0.819
0.805
0.839
0.829
0.828
0.827
0.801
0.829
0.81
0.824
0.821
0.805
0.823
0.821
0.822
0.816
0.803
0.807
0.798
0.793
0.797
0.817
0.781
0.797
0.822
0.813
0.825
0.819
0.819
0.818
0.826
0.797
0.816
0.791
0.818
0.817
0.803
0.82
0.815
0.817
0.831
0.822
0.797
0.831
0.806
0.818
0.82
0.823
0.804
0.82


In [35]:
sampleSize = 1000

#test for 120 cases
for i in range (0,120,1):
    test(X_fullTest[i*sampleSize:(i+1)*sampleSize,:], y_fullTest[i*sampleSize:(i+1)*sampleSize], model_shuffle, batchSize, True)

0.269
0.25
0.244
0.244
0.238
0.271
0.277
0.232
0.247
0.281
0.242
0.266
0.272
0.272
0.235
0.27
0.264
0.257
0.242
0.243
0.254
0.265
0.247
0.246
0.237
0.268
0.258
0.253
0.275
0.254
0.265
0.266
0.218
0.237
0.242
0.267
0.261
0.277
0.274
0.278
0.246
0.252
0.232
0.252
0.244
0.255
0.264
0.227
0.254
0.225
0.256
0.267
0.268
0.257
0.247
0.235
0.263
0.236
0.272
0.25
0.27
0.257
0.266
0.24
0.273
0.246
0.247
0.27
0.244
0.263
0.266
0.262
0.281
0.274
0.254
0.267
0.24
0.267
0.245
0.249
0.237
0.279
0.243
0.257
0.254
0.244
0.259
0.246
0.237
0.23
0.27
0.257
0.256
0.273
0.256
0.259
0.245
0.256
0.285
0.25
0.259
0.273
0.28
0.251
0.248
0.257
0.256
0.292
0.234
0.248
0.254
0.271
0.239
0.282
0.237
0.256
0.244
0.259
0.248
0.242
